# Notebook 9: Corner Cases and Edge Cases in Autonomous Driving

**Session 2: Failure Modes and Edge Cases**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/milinpatel07/Autonomous-Driving_AI-Safety-and-Security/blob/main/AV_Perception_Safety_Workshop/Session_2_Failure_Modes_and_Edge_Cases/notebooks/09_Corner_Cases_and_Edge_Cases.ipynb)

**Author:** Milin Patel  
**Duration:** ~20 minutes

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- ✅ Distinguish between edge cases and corner cases
- ✅ Understand the long-tail distribution in autonomous driving
- ✅ Identify corner cases for specific scenarios
- ✅ Apply simulation-based testing for coverage
- ✅ Analyze coverage gaps and prioritize testing
- ✅ Design mitigation strategies for corner cases

---

## 📦 Setup and Imports

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import itertools
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

---

## 1️⃣ Definitions: Edge Cases vs Corner Cases

### Edge Cases

**Definition:** Scenarios at the **boundary** of the Operational Design Domain (ODD)

**Characteristics:**
- Test **single parameter** at its limit
- Well-defined boundaries
- Predictable locations in parameter space

**Examples:**
- Maximum speed (e.g., 60 km/h)
- Minimum visibility (e.g., 50m in fog)
- Maximum rain intensity (e.g., 50 mm/h)
- Steepest grade (e.g., 15% incline)

**Testing:** Systematically test each boundary

---

### Corner Cases

**Definition:** Scenarios involving **multiple simultaneous** edge conditions

**Characteristics:**
- Test **combinations** of parameters
- Rare in real world
- Exponentially many possibilities
- Often overlooked in testing

**Examples:**
- Maximum speed + heavy rain + sharp curve + pedestrian crossing
- Sunset glare + wet road + cyclist + construction zone
- Snow + night + no lane markings + animal on road

**Challenge:** Combinatorial explosion (too many to test exhaustively)

---

### Visual Comparison

In [ ]:
# Visualize edge case vs corner case
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Edge Cases: Single parameter boundaries
ax1 = axes[0]
# Draw ODD region
from matplotlib.patches import Rectangle
rect = Rectangle((0, 0), 60, 50, facecolor='lightgreen', alpha=0.3, label='Safe ODD')
ax1.add_patch(rect)

# Mark edge cases
edge_cases = [
    (60, 25, 'Max Speed'),
    (30, 50, 'Max Rain'),
    (0, 25, 'Min Speed'),
    (30, 0, 'Clear Weather')
]
for x, y, label in edge_cases:
    ax1.scatter(x, y, s=200, c='orange', marker='o', edgecolors='black', linewidth=2, zorder=5)
    ax1.text(x, y+3, label, ha='center', fontsize=9, fontweight='bold')

ax1.set_xlabel('Speed (km/h)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Rain Intensity (mm/h)', fontsize=12, fontweight='bold')
ax1.set_title('Edge Cases: Single Parameter Boundaries', fontsize=13, fontweight='bold')
ax1.set_xlim(-5, 70)
ax1.set_ylim(-5, 60)
ax1.grid(True, alpha=0.3)
ax1.legend()

# Corner Cases: Multiple parameter combinations
ax2 = axes[1]
rect2 = Rectangle((0, 0), 60, 50, facecolor='lightgreen', alpha=0.3, label='Safe ODD')
ax2.add_patch(rect2)

# Mark corner cases
corner_cases = [
    (60, 50, 'Max Speed\n+ Max Rain'),
    (60, 0, 'Max Speed\n+ Clear'),
    (0, 50, 'Min Speed\n+ Max Rain'),
    (0, 0, 'Min Speed\n+ Clear')
]
for x, y, label in corner_cases:
    ax2.scatter(x, y, s=200, c='red', marker='X', edgecolors='black', linewidth=2, zorder=5)
    ax2.text(x+3, y+3, label, ha='left', fontsize=8, fontweight='bold')

ax2.set_xlabel('Speed (km/h)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Rain Intensity (mm/h)', fontsize=12, fontweight='bold')
ax2.set_title('Corner Cases: Multiple Boundaries', fontsize=13, fontweight='bold')
ax2.set_xlim(-5, 70)
ax2.set_ylim(-5, 60)
ax2.grid(True, alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()

print("\n💡 Key Insight:")
print("   - Edge cases: Test boundaries one at a time")
print("   - Corner cases: Test combinations (corners of parameter space)")

---

## 2️⃣ The Long-Tail Distribution in Autonomous Driving

### The 80-20 Rule (Pareto Principle)

In autonomous driving:
- **80% of miles:** Common, routine scenarios
- **20% of miles:** Account for **80% of safety-critical events**

**Implication:** Cannot ignore the long tail!

### Long-Tail Phenomenon

**Head (Common):**
- Clear weather, daytime
- Highway or urban roads
- Standard vehicles (cars, trucks)
- Good road conditions

**Tail (Rare but Critical):**
- Unusual weather (sandstorm, ice storm)
- Rare objects (escaped livestock, furniture on road)
- Infrastructure edge cases (missing signs, construction)
- Compound scenarios (multiple simultaneous challenges)

**Challenge for ML:** Models trained on common data perform poorly on tail

In [ ]:
# Visualize long-tail distribution
scenarios = ['Clear\nHighway', 'Urban\nDay', 'Light\nRain', 'Night\nDriving', 'Heavy\nRain', 
             'Fog', 'Snow', 'Construction', 'Unusual\nObject', 'Extreme\nWeather']
frequencies = [10000, 5000, 2000, 1500, 500, 200, 100, 50, 20, 5]
risk_levels = [1, 2, 3, 3, 5, 6, 7, 7, 9, 10]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))

# Plot 1: Frequency distribution
colors = ['green'] * 4 + ['yellow'] * 3 + ['red'] * 3
bars = ax1.bar(scenarios, frequencies, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
ax1.set_ylabel('Frequency (miles driven)', fontsize=12, fontweight='bold')
ax1.set_title('Long-Tail Distribution: Scenario Frequency', fontsize=14, fontweight='bold')
ax1.set_yscale('log')
ax1.grid(True, alpha=0.3, axis='y')

# Annotate head vs tail
ax1.axvspan(-0.5, 3.5, alpha=0.2, color='green', label='Head (Common)')
ax1.axvspan(3.5, 9.5, alpha=0.2, color='red', label='Tail (Rare)')
ax1.legend(loc='upper right', fontsize=11)

# Plot 2: Risk level
ax2.plot(scenarios, risk_levels, marker='o', markersize=10, linewidth=3, color='darkred')
ax2.fill_between(range(len(scenarios)), risk_levels, alpha=0.3, color='red')
ax2.set_ylabel('Risk Level (1-10)', fontsize=12, fontweight='bold')
ax2.set_xlabel('Scenario Type', fontsize=12, fontweight='bold')
ax2.set_title('Risk Increases in the Tail', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 11)

plt.tight_layout()
plt.show()

print("\n⚠️ Key Observation:")
print("   - Rare scenarios (long tail) have HIGHEST risk")
print("   - Cannot achieve safety by only testing common cases")
print("   - Must actively seek out and test corner cases")

---

## 3️⃣ Real-World Corner Case Examples

### Categories of Corner Cases

**1. Environmental:**
- Sandstorm in Arizona
- Black ice (invisible ice on road)
- Flooding with submerged lane markings
- Direct sunlight causing camera saturation

**2. Infrastructure:**
- Missing or faded lane markings
- Contradictory traffic signs
- Construction zones with temporary signals
- Potholes and road damage

**3. Traffic Participants:**
- Escaped farm animals
- Oversized loads (wide trucks)
- Emergency vehicles with unusual maneuvers
- Pedestrians in unexpected locations

**4. Sensor-Related:**
- Dirty/occluded cameras or LiDAR
- Sensor calibration drift
- Reflections (wet roads, glass buildings)
- Laser interference (multiple LiDARs)

**5. Compound Scenarios:**
- Night + rain + construction + pedestrian
- Fog + narrow road + oncoming truck + cyclist
- Snow + no GPS + unclear lane markings + intersection

In [ ]:
# Real-world corner case database
corner_cases = pd.DataFrame({
    'Category': [
        'Environmental', 'Environmental', 'Environmental', 'Environmental',
        'Infrastructure', 'Infrastructure', 'Infrastructure', 'Infrastructure',
        'Traffic', 'Traffic', 'Traffic', 'Traffic',
        'Sensor', 'Sensor', 'Sensor',
        'Compound', 'Compound', 'Compound'
    ],
    'Scenario': [
        'Sandstorm', 'Black ice', 'Flooding', 'Direct sun glare',
        'Missing lane markings', 'Contradictory signs', 'Construction zone', 'Large potholes',
        'Escaped animals', 'Oversized load', 'Emergency vehicle', 'Jaywalking at night',
        'Dirty camera', 'LiDAR interference', 'Wet road reflections',
        'Night+rain+construction', 'Fog+narrow road+truck', 'Snow+no GPS+unclear lanes'
    ],
    'Frequency': [
        'Very rare', 'Rare', 'Rare', 'Common',
        'Common', 'Rare', 'Common', 'Common',
        'Very rare', 'Rare', 'Rare', 'Rare',
        'Common', 'Rare', 'Common',
        'Rare', 'Rare', 'Very rare'
    ],
    'Risk': [
        'Critical', 'Critical', 'High', 'High',
        'High', 'High', 'High', 'Medium',
        'Critical', 'High', 'High', 'Critical',
        'Medium', 'Medium', 'Medium',
        'Critical', 'Critical', 'Critical'
    ],
    'Tested': [
        '❌', '⚠️', '⚠️', '✅',
        '✅', '❌', '✅', '✅',
        '❌', '⚠️', '✅', '⚠️',
        '✅', '⚠️', '✅',
        '❌', '❌', '❌'
    ]
})

display(corner_cases)

# Count gaps
not_tested = corner_cases[corner_cases['Tested'] == '❌']
critical_not_tested = not_tested[not_tested['Risk'] == 'Critical']

print(f"\n⚠️ Testing Gaps:")
print(f"   Total scenarios: {len(corner_cases)}")
print(f"   Not fully tested: {len(corner_cases[corner_cases['Tested'] != '✅'])}")
print(f"   Critical risk, not tested: {len(critical_not_tested)}")
print(f"\n   Critical gaps: {list(critical_not_tested['Scenario'])}")

In [ ]:
# Visualize risk vs frequency matrix
risk_map = {'Low': 1, 'Medium': 2, 'High': 3, 'Critical': 4}
freq_map = {'Very rare': 1, 'Rare': 2, 'Common': 3}

corner_cases['Risk_Score'] = corner_cases['Risk'].map(risk_map)
corner_cases['Freq_Score'] = corner_cases['Frequency'].map(freq_map)

# Create risk matrix
fig, ax = plt.subplots(figsize=(12, 8))

# Color by testing status
test_colors = {'✅': 'green', '⚠️': 'orange', '❌': 'red'}
colors = [test_colors[t] for t in corner_cases['Tested']]

scatter = ax.scatter(corner_cases['Freq_Score'], corner_cases['Risk_Score'], 
                     c=colors, s=300, alpha=0.7, edgecolors='black', linewidth=2)

# Add labels for critical untested cases
for idx, row in critical_not_tested.iterrows():
    ax.annotate(row['Scenario'], 
                (row['Freq_Score'], row['Risk_Score']),
                xytext=(10, 10), textcoords='offset points',
                fontsize=9, fontweight='bold',
                bbox=dict(boxstyle='round,pad=0.5', facecolor='red', alpha=0.3),
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))

# Divide into quadrants
ax.axvline(x=2, color='gray', linestyle='--', linewidth=2, alpha=0.5)
ax.axhline(y=2.5, color='gray', linestyle='--', linewidth=2, alpha=0.5)

# Label quadrants
ax.text(1.5, 3.5, 'Low Priority\n(Rare, Low Risk)', ha='center', fontsize=10, 
        bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.3))
ax.text(2.5, 3.5, 'Medium Priority\n(Common, High Risk)', ha='center', fontsize=10,
        bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.3))
ax.text(1.5, 1.5, 'Very Low Priority', ha='center', fontsize=10,
        bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.3))
ax.text(2.5, 1.5, 'Monitor', ha='center', fontsize=10,
        bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.3))

# Add HIGH PRIORITY zone
from matplotlib.patches import Rectangle
high_priority = Rectangle((0.5, 3), 1.5, 1, facecolor='red', alpha=0.1, linewidth=3, 
                          edgecolor='red', linestyle='--')
ax.add_patch(high_priority)
ax.text(1.25, 3.9, 'HIGH PRIORITY!\n(Rare + Critical)', ha='center', fontsize=11, 
        fontweight='bold', color='darkred')

ax.set_xlabel('Frequency', fontsize=12, fontweight='bold')
ax.set_ylabel('Risk Level', fontsize=12, fontweight='bold')
ax.set_title('Corner Case Risk Matrix', fontsize=14, fontweight='bold')
ax.set_xticks([1, 2, 3])
ax.set_xticklabels(['Very Rare', 'Rare', 'Common'])
ax.set_yticks([1, 2, 3, 4])
ax.set_yticklabels(['Low', 'Medium', 'High', 'Critical'])
ax.set_xlim(0.5, 3.5)
ax.set_ylim(0.5, 4.5)
ax.grid(True, alpha=0.3)

# Legend
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor='green', markersize=10, label='Tested'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='orange', markersize=10, label='Partially Tested'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label='Not Tested')
]
ax.legend(handles=legend_elements, loc='lower right', fontsize=11)

plt.tight_layout()
plt.show()

print("\n💡 Priority: Focus testing on HIGH PRIORITY zone (rare + critical + untested)")

---

## 4️⃣ Combinatorial Explosion Problem

**Challenge:** Number of corner cases grows exponentially with parameters

**Example:** Campus shuttle with 5 parameters, each with 3 values:
- Weather: {Clear, Rain, Fog}
- Time: {Day, Dusk, Night}
- Traffic: {Light, Medium, Heavy}
- Road: {Dry, Wet, Icy}
- Obstacles: {None, Pedestrian, Construction}

**Total combinations:** 3^5 = **243 scenarios**

**With 10 parameters:** 3^10 = **59,049 scenarios**

**Impractical to test exhaustively!**

In [ ]:
# Demonstrate combinatorial explosion
parameters = {
    'Weather': ['Clear', 'Rain', 'Fog'],
    'Time': ['Day', 'Dusk', 'Night'],
    'Traffic': ['Light', 'Medium', 'Heavy'],
    'Road': ['Dry', 'Wet', 'Icy'],
    'Obstacles': ['None', 'Pedestrian', 'Construction']
}

# Generate all combinations
all_combinations = list(itertools.product(*parameters.values()))

print(f"Parameter Space:")
for param, values in parameters.items():
    print(f"  {param}: {len(values)} values - {values}")

print(f"\nTotal combinations: {len(all_combinations)}")
print(f"\nSample scenarios:")
for i, combo in enumerate(all_combinations[:10]):
    scenario = ', '.join([f"{p}:{v}" for p, v in zip(parameters.keys(), combo)])
    print(f"  {i+1}. {scenario}")

print(f"\n  ... and {len(all_combinations) - 10} more!")

# Visualize growth
num_params = range(1, 11)
combinations = [3**n for n in num_params]

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(num_params, combinations, marker='o', markersize=10, linewidth=3, color='darkred')
ax.fill_between(num_params, combinations, alpha=0.3, color='red')
ax.set_xlabel('Number of Parameters', fontsize=12, fontweight='bold')
ax.set_ylabel('Number of Combinations', fontsize=12, fontweight='bold')
ax.set_title('Combinatorial Explosion: Corner Cases', fontsize=14, fontweight='bold')
ax.set_yscale('log')
ax.grid(True, alpha=0.3)

# Annotate
ax.annotate(f'{combinations[4]} combinations', xy=(5, combinations[4]), 
            xytext=(6, combinations[6]), fontsize=11, fontweight='bold',
            arrowprops=dict(arrowstyle='->', lw=2))

plt.tight_layout()
plt.show()

print("\n⚠️ Problem: Cannot test all combinations!")
print("   Solution: Prioritized, risk-based testing")

---

## 5️⃣ Simulation-Based Testing

**Solution:** Use simulation to generate and test corner cases

### Advantages of Simulation
1. **Scalability:** Test millions of scenarios
2. **Safety:** No risk to humans or property
3. **Reproducibility:** Exact scenario replay
4. **Coverage:** Systematically explore parameter space
5. **Cost:** Cheaper than real-world testing

### Limitations
1. **Sim-to-real gap:** Simulation ≠ reality
2. **Sensor modeling:** Difficult to model perfectly
3. **Unexpected events:** Can't simulate unknown unknowns
4. **Validation:** Still need real-world testing

### Best Practice
**Hybrid approach:**
1. Simulation for broad coverage
2. Real-world for validation
3. Continuous update based on real incidents

In [ ]:
# Simulation-based coverage analysis
def generate_test_scenarios(num_scenarios=1000):
    """Generate random test scenarios"""
    scenarios = []
    for _ in range(num_scenarios):
        scenario = {
            'weather': np.random.choice(['Clear', 'Rain', 'Fog'], p=[0.7, 0.2, 0.1]),
            'time': np.random.choice(['Day', 'Dusk', 'Night'], p=[0.6, 0.2, 0.2]),
            'traffic': np.random.choice(['Light', 'Medium', 'Heavy'], p=[0.5, 0.3, 0.2]),
            'road': np.random.choice(['Dry', 'Wet', 'Icy'], p=[0.7, 0.25, 0.05]),
            'obstacles': np.random.choice(['None', 'Pedestrian', 'Construction'], p=[0.8, 0.15, 0.05])
        }
        scenarios.append(scenario)
    return scenarios

# Generate scenarios
sim_scenarios = generate_test_scenarios(10000)

# Compute coverage
total_possible = len(all_combinations)
unique_tested = set()
for s in sim_scenarios:
    combo = tuple(s.values())
    unique_tested.add(combo)

coverage = len(unique_tested) / total_possible * 100

print(f"Simulation Results:")
print(f"  Total scenarios simulated: {len(sim_scenarios)}")
print(f"  Unique scenarios covered: {len(unique_tested)}")
print(f"  Total possible scenarios: {total_possible}")
print(f"  Coverage: {coverage:.1f}%")

# Identify untested corner cases
untested = set(all_combinations) - unique_tested
print(f"\n  Untested corner cases: {len(untested)}")

# Analyze untested cases
untested_df = pd.DataFrame(list(untested), columns=parameters.keys())
print(f"\n  Characteristics of untested cases:")
for col in untested_df.columns:
    print(f"    {col}: {untested_df[col].value_counts().to_dict()}")

print("\n💡 Observation: Rare combinations are under-tested (Icy roads, Fog, etc.)")
print("   Solution: Targeted testing of these specific corner cases")

---

## 6️⃣ Campus Shuttle Case Study

**Scenario:** Design corner case testing for a Level 4 campus autonomous shuttle

**ODD:**
- Geography: University campus roads only
- Speed: Maximum 25 km/h
- Time: 7 AM - 10 PM
- Weather: Clear to light rain
- Roads: Paved, marked lanes

**Task:** Identify potential corner cases

In [ ]:
# Campus shuttle corner case identification
campus_corner_cases = pd.DataFrame({
    'ID': range(1, 16),
    'Corner_Case': [
        'Rush hour + event (football game)',
        'Night + fog + pedestrians jaywalking',
        'Construction zone + temporary signs',
        'Heavy rain + puddles obscuring lanes',
        'Bike lane merge + group of cyclists',
        'Delivery truck parked blocking lane',
        'Scooters/skateboards in roadway',
        'Emergency vehicle approaching',
        'Fallen tree branch after storm',
        'Campus police directing traffic',
        'Graduation ceremony crowd crossing',
        'Snow day + no GPS signal',
        'Sunset glare + wet road reflections',
        'Multiple shuttles at same stop',
        'Dog off-leash running into road'
    ],
    'Likelihood': ['High', 'Medium', 'High', 'Medium', 'Medium', 'High', 'High', 'Low', 
                   'Low', 'Low', 'Low', 'Low', 'Medium', 'Medium', 'Low'],
    'Severity': ['High', 'Critical', 'Medium', 'Medium', 'High', 'Medium', 'High', 'High',
                 'Medium', 'High', 'Critical', 'High', 'Medium', 'Low', 'Critical'],
    'Currently_Tested': ['✅', '❌', '✅', '⚠️', '❌', '✅', '❌', '⚠️', '❌', '❌', 
                         '❌', '❌', '⚠️', '✅', '❌']
})

display(campus_corner_cases)

# Risk prioritization
risk_scores = {
    ('Low', 'Low'): 1, ('Low', 'Medium'): 2, ('Low', 'High'): 3, ('Low', 'Critical'): 4,
    ('Medium', 'Low'): 2, ('Medium', 'Medium'): 4, ('Medium', 'High'): 6, ('Medium', 'Critical'): 8,
    ('High', 'Low'): 3, ('High', 'Medium'): 6, ('High', 'High'): 9, ('High', 'Critical'): 12
}

campus_corner_cases['Risk_Score'] = campus_corner_cases.apply(
    lambda row: risk_scores[(row['Likelihood'], row['Severity'])], axis=1
)

# Sort by risk and testing status
untested = campus_corner_cases[campus_corner_cases['Currently_Tested'] == '❌']
high_risk_untested = untested[untested['Risk_Score'] >= 8].sort_values('Risk_Score', ascending=False)

print("\n🚨 HIGH PRIORITY: Test these corner cases immediately:\n")
for idx, row in high_risk_untested.iterrows():
    print(f"  {row['ID']}. {row['Corner_Case']}")
    print(f"      Likelihood: {row['Likelihood']}, Severity: {row['Severity']}, Risk Score: {row['Risk_Score']}\n")

In [ ]:
# Visualize risk matrix for campus shuttle
likelihood_map = {'Low': 1, 'Medium': 2, 'High': 3}
severity_map = {'Low': 1, 'Medium': 2, 'High': 3, 'Critical': 4}
test_colors = {'✅': 'green', '⚠️': 'orange', '❌': 'red'}

fig, ax = plt.subplots(figsize=(12, 8))

for idx, row in campus_corner_cases.iterrows():
    x = likelihood_map[row['Likelihood']]
    y = severity_map[row['Severity']]
    color = test_colors[row['Currently_Tested']]
    
    ax.scatter(x, y, s=400, c=color, alpha=0.7, edgecolors='black', linewidth=2)
    ax.text(x, y, str(row['ID']), ha='center', va='center', 
            fontsize=10, fontweight='bold', color='white')

# Grid
ax.set_xlabel('Likelihood', fontsize=13, fontweight='bold')
ax.set_ylabel('Severity', fontsize=13, fontweight='bold')
ax.set_title('Campus Shuttle: Corner Case Risk Matrix', fontsize=15, fontweight='bold')
ax.set_xticks([1, 2, 3])
ax.set_xticklabels(['Low', 'Medium', 'High'])
ax.set_yticks([1, 2, 3, 4])
ax.set_yticklabels(['Low', 'Medium', 'High', 'Critical'])
ax.set_xlim(0.5, 3.5)
ax.set_ylim(0.5, 4.5)
ax.grid(True, alpha=0.3)

# Highlight high-risk zone
from matplotlib.patches import Polygon
high_risk_zone = Polygon([(2, 3), (3, 3), (3, 4), (2, 4)], 
                         facecolor='red', alpha=0.15, edgecolor='red', 
                         linewidth=3, linestyle='--')
ax.add_patch(high_risk_zone)
ax.text(2.5, 3.7, 'HIGH RISK\nPRIORITY', ha='center', fontsize=12, 
        fontweight='bold', color='darkred')

# Legend
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor='green', markersize=12, label='Tested'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='orange', markersize=12, label='Partially Tested'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=12, label='Not Tested')
]
ax.legend(handles=legend_elements, loc='upper left', fontsize=11)

plt.tight_layout()
plt.show()

print("\n📋 Action Plan:")
print("   1. Immediately test red dots in high-risk zone")
print("   2. Develop mitigation strategies for each")
print("   3. Add to simulation test suite")
print("   4. Validate in real-world controlled tests")

---

## ✏️ Exercise: Corner Case Identification

**Task:** For the campus shuttle, identify additional corner cases and prioritize testing.

**Consider:**
1. What combinations of conditions create corner cases?
2. Which are most likely to cause failures?
3. What mitigation strategies would you implement?
4. How would you test these in simulation vs real world?

In [ ]:
# TODO: Add your corner cases

your_corner_cases = pd.DataFrame({
    'Corner_Case': [
        # Add your identified corner cases
        # Example: 'Fireworks display + crowd + low visibility'
    ],
    'Likelihood': [],  # Low, Medium, or High
    'Severity': [],    # Low, Medium, High, or Critical
    'Mitigation_Strategy': []  # How to handle this case?
})

# Display your analysis
if len(your_corner_cases) > 0:
    display(your_corner_cases)
else:
    print("Add your corner cases above!")

print("\n🤔 Reflection Questions:")
print("   1. Which corner cases are hardest to test?")
print("   2. What sensors/algorithms would help detect these?")
print("   3. What is the safety response for each case?")
print("   4. How do you balance coverage vs testing resources?")

---

## 🎯 Key Takeaways

### Definitions
- **Edge cases:** Boundary of single parameter
- **Corner cases:** Combination of multiple edge conditions
- **Long tail:** Rare scenarios with high risk

### The Challenge
- **Combinatorial explosion:** Cannot test exhaustively
- **Long-tail risk:** Rare scenarios cause most accidents
- **ML weakness:** Poor generalization to untrained scenarios

### Solutions
1. **Risk-based prioritization:** Focus on high likelihood × high severity
2. **Simulation:** Broad coverage at scale
3. **Real-world validation:** Confirm simulation results
4. **Continuous learning:** Update based on real incidents

### Best Practices
1. ✅ **Systematically identify** corner cases for your ODD
2. ✅ **Prioritize by risk:** Test critical cases first
3. ✅ **Use simulation** for broad coverage
4. ✅ **Design mitigations** for each corner case
5. ✅ **Monitor in deployment:** Detect and log corner cases
6. ✅ **Iterate:** Continuously expand test coverage

### Connection to Safety
- **ISO 21448 SOTIF:** Systematic identification of unsafe scenarios
- **Coverage analysis:** Demonstrate testing completeness
- **Risk mitigation:** Design system to handle corner cases gracefully

---

## 🔜 Next: Adversarial Attacks on Perception

Now let's explore intentional attacks on AV perception systems!

**Ready?** Open `10_Adversarial_Attacks_on_Perception.ipynb`

---

*Notebook created by Milin Patel | Hochschule Kempten*  
*Last updated: 2025-01-18*